In [1]:
import progres as pg

In [26]:
import pandas as pd
import numpy as np
import os
import Bio.PDB as bpdb

In [36]:
class ResSelect(bpdb.Select):
    global start_res
    global end_res
    def accept_residue(self, res):
        if res.id[1] >= start_res and res.id[1] <= end_res and res.parent.id == chain_id:
            return False
        else:
            return True

In [44]:
def select_nterm(prot_path):
    struc = bpdb.PDBParser().get_structure('temp', 'AlphaFold2_models/GTs_best/AFJ52996.1/AFJ52996.1_ranked_0.pdb')
    prot_len = len(struc[0]['A'])
    start_res=int(prot_len/2)
    end_res=prot_len
    chain_id = 'A'
    nterm = bpdb.PDBIO()
    nterm.set_structure(struc)
    nterm.save('temp.pdb', ResSelect())


In [45]:
rep_df = pd.DataFrame(columns=np.arange(128))
rep_Nt_df = pd.DataFrame(columns=np.arange(128))
for path, dirs, files in os.walk('AlphaFold2_models/GTs_best/'):
    if len(dirs) == 0:
        prot_name = os.path.basename(path)
        rep = pg.embed_structure(path+'/'+prot_name+'_ranked_0.pdb')
        rep_df.loc[prot_name] = rep.detach().numpy()
        select_nterm(path+'/'+prot_name+'_ranked_0.pdb')
        rep = pg.embed_structure('temp.pdb')
        rep_Nt_df.loc[prot_name] = rep.detach().numpy()

In [48]:
rep_df.index.names = ['enzyme']
rep_df.to_csv('../encodings/progres.tsv',sep='\t')
rep_Nt_df.index.names = ['enzyme']
rep_Nt_df.to_csv('../encodings/progres_Nt.tsv',sep='\t')